## Initial set-up

In [2]:
pip install duckdb

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas 

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install ipython-sql

In [5]:
pip install SQLAlchemy

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install duckdb-engine

Note: you may need to restart the kernel to use updated packages.


In [7]:
import duckdb
import pandas as pd
import sqlalchemy
# No need to import duckdb_engine
#  SQLAlchemy will auto-detect the driver needed based on your connection string!

# Import ipython-sql Jupyter extension to create SQL cells
%load_ext sql

In [8]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

## Instantiate duckdb in-memory

In [9]:
%sql duckdb:///:memory:

## Install JSON extension

Since the FHIR resources are stored as JSON in a column and we want to query parts of the JSON we install the JSON extension.

In [10]:
%%sql

install 'json';
load 'json';

""


DuckDB can directly query parquet files with full support for SQL. No import of data necessary !

In [12]:
%%sql
select count(*) from 'resource-snappy.parquet';

,count_star()
0,374130


In [22]:
%%sql
select * from 'resource-snappy.parquet' using sample 100;

,id,resourceType,resourceJson
0,1013cd60-5b76-4172-b341-1cdb737ed03a,Observation,"{""resourceType"":""Observation"",""id"":""1013cd60-5..."
1,0fcc56e0-d83d-4bcd-8f30-9c80dc001c44,Observation,"{""resourceType"":""Observation"",""id"":""0fcc56e0-d..."
2,76db7004-db0e-47ff-a1e7-6263c31b4cf1,MedicationRequest,"{""resourceType"":""MedicationRequest"",""id"":""76db..."
3,f7bc081c-60c2-4efc-9d45-d4755a83a35b,Encounter,"{""resourceType"":""Encounter"",""id"":""f7bc081c-60c..."
4,7dcee691-5e87-4f3d-ad49-58e425f842c9,Claim,"{""resourceType"":""Claim"",""id"":""7dcee691-5e87-4f..."
...,...,...,...
95,243604f3-6807-433e-bd94-016da7dd2238,CarePlan,"{""resourceType"":""CarePlan"",""id"":""243604f3-6807..."
96,230712a1-f819-4511-90a8-af33f3f9b3fa,Claim,"{""resourceType"":""Claim"",""id"":""230712a1-f819-45..."
97,b922a762-27df-4c4d-8df6-49f1e8b2a1b4,Observation,"{""resourceType"":""Observation"",""id"":""b922a762-2..."
98,6731ccf5-1133-4d1b-91cc-7eaf41cef2d4,Encounter,"{""resourceType"":""Encounter"",""id"":""6731ccf5-113..."


For convinience lets create a resource table and then import the parquet file into this table and then use the table to run SQL queries.

In [14]:
%%sql
create table resource(id VARCHAR, resourceType VARCHAR, resourceJson JSON);

""


In [18]:
%%sql
INSERT INTO resource SELECT * FROM read_parquet('resource-snappy.parquet');

,Count
0,374130


In [19]:
%%sql
select count(*) from resource where json_extract_string(resourceJson, '$.resourceType') = 'Observation' and json_extract_string(resourceJson, '$.category[0].coding[0].code') = 'laboratory';

,count_star()
0,117050


In [21]:
%%sql
select unnest(from_json(json_extract(resourceJson, '$.code.coding'), '[{"system":"VARCHAR", "code":"VARCHAR", "display":"VARCHAR"}]')) from resource using sample 100;

,"unnest(from_json(json_extract(""resourceJson"", '$.code.coding'), '[{""system"":""VARCHAR"", ""code"":""VARCHAR"", ""display"":""VARCHAR""}]'))"
0,"{'system': 'http://loinc.org', 'code': '29463-..."
1,"{'system': 'http://loinc.org', 'code': '72514-..."
2,"{'system': 'http://loinc.org', 'code': '29463-..."
3,"{'system': 'http://loinc.org', 'code': '1920-8..."
4,"{'system': 'http://loinc.org', 'code': '6299-2..."
...,...
61,"{'system': 'http://loinc.org', 'code': '29463-..."
62,"{'system': 'http://loinc.org', 'code': '59576-..."
63,"{'system': 'http://loinc.org', 'code': '2947-0..."
64,"{'system': 'http://loinc.org', 'code': '55284-..."


## More to come...